# Load data

In [ ]:
import pandas as pd

# 1. Load the CSV file
# NOTE: Make sure the file path is correct for your environment
#df = pd.read_csv("D:\Documents\SARA\MIYARU\citizen science\miyaru cs apr25.csv")
df = pd.read_csv("/Volumes/AugustMV25B/miyaru cs apr25.csv")
# 2. Check the existing column names
print("Old column names:")
print(df.columns)

# Clean up
## Change columnnames

In [ ]:
df.columns = ['Submission ID', 'Last updated', 'Submission started', 'Status',
       'Current step', 'Name',
       'Affiliation',
       'Email',
       'Updates_required',
       'Species',
       'Other_species',
       'Number',
       'Depth_range',
       'Size',
       'Behaviours',
       'Injuries',
       'Mating_scars',
       'Notes',
       'Media',
       'Atoll', 'Island',
       'Site_name',
       'Coords',
       'Habitat',
       'Activity',
       'Encounter_date',
       'Start_time',
       'Duration',
       'Surface_temperature',
       'Bottom_temperature',
       'Visibility',
       'Tide', 'Current_strength',
       'Current_direction',
       'Water_movement',
       'N_people',
       'N_Boats',
       'Bait',
       'Bait_type',
       'Bait_composition',
       'Bait_amount',
       'Notes_2',
       'Errors', 'Url', 'Network ID']

# 4. Verify the columns have been renamed
print("\nNew column names:")
print(df.columns)


## Add UniqueID

In [ ]:

###############################################################################
# 1. DICTIONARIES OF ABBREVIATIONS
###############################################################################
species_to_abbr = {
    "Blacktip Reef Shark (Carcharhinus melanopterus)": "BT",
    "Whitetip Reef Shark (Triaenodon obesus)": "WT",
    "Grey Reef Shark (Carcharinus amblyrhynchos)": "Gr",
    "Silvertip Shark (Carcharhinus albimarginatus)": "ST",
    "Nurse Shark (Nebrius ferrugineus)": "Nu",
    "Zebra Shark (Stegostoma fasciatum)": "Ze",
    "Leopard/Zebra Shark (Stegostoma fasciatum)": "Ze",
    "Spinner Shark (Carcharhinus brevipinna)": "Sp",
    "Tiger Shark (Galeocerdo cuvier)": "Ti",
    "Bull Shark (Carcharhinus leucas)": "Bu",
    "Lemon Shark (Negaprion acutidens)": "Le",
    "Great Hammerhead Shark (Sphyrna mokarran)": "GH",
    "Scalloped Hammerhead Shark (Sphyrna lewini)": "SH",
    "Pelagic Thresher Shark (Alopias pelagicus)": "PT",
    "Bigeye Thresher Shark (Alopias superciliosus)": "BET",
    "Oceanic Whitetip Shark (Carcharhinus longimanus)": "OW",
    "Shortfin Mako Shark (Isurus oxyrinchus)": "SM",
    "Smalltooth Sand Tiger Shark (Odantaspis ferox)": "SS",
    "Giant Guitarfish (Rhynchobatus djiddensis)": "GG",
    "Bowmouth Guitarfish (Rhina ancylostoma)": "BG",
    "Other": "Other"
}

atoll_to_abbr = {
    "Haa Alif": "HA",
    "Haa Dhaal": "HDh",
    "Shaviyani": "Sh",
    "Noonu": "N",
    "Lhavhiyani": "Lh",
    "Raa": "R",
    "Baa": "B",
    "Kaafu": "K",
    "Alif Alif": "AA",
    "Alif Dhaal": "ADh",
    "Vaavu": "V",
    "Dhaalu": "Dh",
    "Meemu": "M",
    "Thaa": "T",
    "Laamu": "L",
    "Gaaf Alif": "GA",
    "Gaaf Dhaal": "GDh",
    "Gnaviyani (Fuvahmulah)": "Gn",
    "Seenu (Addu)": "S",
}

##########################################################################
# 2) Convert Encounter_date to datetime and create yyyymmdd column
##########################################################################
df["Encounter_date"] = pd.to_datetime(df["Encounter_date"], errors='coerce')
df["Date_yyyymmdd"] = df["Encounter_date"].dt.strftime("%Y%m%d")

##########################################################################
# 3) Map species and atoll columns to abbreviations
##########################################################################
df["Species_Abbr"] = df["Species"].map(species_to_abbr)
df["Atoll_Abbr"] = df["Atoll"].map(atoll_to_abbr)

##########################################################################
# 4) Build the grouping-based counter
#    For each unique (Species, Atoll, Encounter_date), 
#    assign an incrementing counter: 1, 2, 3...
##########################################################################
# Drop any rows lacking Species, Atoll, or Encounter_date
df.dropna(subset=["Species", "Atoll", "Encounter_date"], inplace=True)

# Now group and cumcount
df["n"] = (
    df.groupby(["Species", "Atoll", "Encounter_date"])["Encounter_date"]
    .cumcount()
    + 1
)

# This will now work without error
df["n"] = df["n"].astype(int)
##########################################################################
# 5) Construct the Unique_ID column
##########################################################################
df["Unique_ID"] = (
    df["Species_Abbr"].astype(str)
    + "_"
    + df["Atoll_Abbr"].astype(str)
    + "_"
    + df["Date_yyyymmdd"].astype(str)
    + "_"
    + df["n"].astype(str)
)
##########################################################################
# 6) Remove Name = fake fake row
##########################################################################
df = df[df["Name"] != "fake fake"]
# (Optional) See how it looks
df

## Ad latitude and Longitude columns

In [ ]:
import pandas as pd, numpy as np, re

# --- helper ---------------------------------------------------------------
def _to_float(token: str):
    """
    Turn a single latitude or longitude token into a signed float.
      • Accepts optional leading +/‑, trailing N/S/E/W, and embedded °.
    """
    m = re.match(r'\s*([+-]?\d+(?:\.\d+)?)\s*(?:°?\s*([NSEW]))?\s*$', token, re.I)
    if not m:                     # unparsable -> NaN
        return np.nan
    value = float(m.group(1))
    hemi  = (m.group(2) or '').upper()
    if hemi in ('S', 'W'):        # south & west are negative
        value = -value
    return value

def parse_coord(cell):
    """
    Convert any of your three formats to a pd.Series([lat, lon]).
    Works transparently with NaNs.
    """
    if pd.isna(cell):
        return pd.Series([np.nan, np.nan])
    
    s = str(cell).strip()
    if s.startswith('(') and s.endswith(')'):   # strip surrounding parentheses
        s = s[1:-1]
    
    # drop all explicit degree symbols, then split at the comma
    parts = [p.strip() for p in s.replace('°', '').split(',')]
    if len(parts) != 2:
        return pd.Series([np.nan, np.nan])      # malformed
    
    return pd.Series([_to_float(parts[0]), _to_float(parts[1])])

# --- apply ---------------------------------------------------------------
df[['lat', 'lon']] = df['Coords'].apply(parse_coord)


## Create map with locations

In [ ]:
!pip install folium shapely

In [ ]:
# 0)  One‑off install  – run in a Jupyter cell or your terminal
# !pip install folium shapely  # (shapely only if you use CircleMarker)

import folium
from folium.plugins import MarkerCluster   # nice to have
import pandas as pd

# --- assume your df already has numeric columns 'lat' and 'lon' ----------
# df[['lat','lon']] = df['Coords'].apply(parse_coord)   # from previous step

# 1)  Drop rows without coordinates
pts = df.dropna(subset=['lat', 'lon']).copy()

# 2)  Choose a sensible centre & zoom
center = [pts['lat'].mean(), pts['lon'].mean()]

m = folium.Map(
        location=center,
        zoom_start=10,                 # change if you want wider/narrower view
        tiles='Esri.WorldImagery',     # <‑‑ high‑res satellite imagery
        attr='Esri'                    # keeps the credit line
     )

# 3)  Put the points on the map -------------------------------------------
cluster = MarkerCluster().add_to(m)    # comment this out if you don’t want clustering

for _, row in pts.iterrows():
    folium.Marker(
        location=[row.lat, row.lon],
        popup=f"({row['Unique_ID']}: {row.lat:.4f}, {row.lon:.4f})"   # or add any other info here
    ).add_to(cluster)

# 4)  Display inside Jupyter
m

# 5)  …or save to file and open in a browser
m.save('coords_satellite_map.html')
print("✅  Map saved as coords_satellite_map.html")

In [ ]:
df.columns

In [ ]:
df['Coords']

In [ ]:
df['Coords']

In [ ]:
df['Species'].unique()
